## recode "build neural network from scratch"

## sentiment analysis on dou-ban

In [1]:
import pandas as pd 
import jieba
import gensim
import tensorflow as tf
import numpy as np
from tqdm import tqdm
import time

In [2]:
csv_file = "nlp/datasource/movie_comments.csv"
df = pd.read_csv(csv_file)
df.head()

/home/chihao/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,link,name,comment,star
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2
2,3,https://movie.douban.com/subject/26363254/,战狼2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...,2
3,4,https://movie.douban.com/subject/26363254/,战狼2,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4
4,5,https://movie.douban.com/subject/26363254/,战狼2,中二得很,1


In [3]:
df.describe()

,id,link,name,comment,star
count,261497,261497,261497,261495,261497
unique,260150,2761,2760,213970,11
top,10,https://movie.douban.com/subject/1849031/,当幸福来敲门 The Pursuit of Happyness,经典,4
freq,6,396,396,200,43002


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261497 entries, 0 to 261496
Data columns (total 5 columns):
id         261497 non-null object
link       261497 non-null object
name       261497 non-null object
comment    261495 non-null object
star       261497 non-null object
dtypes: object(5)
memory usage: 10.0+ MB


In [5]:
comment = df["comment"]
labels = df["star"]

In [6]:
print(time.perf_counter())
cut_words = comment.apply(lambda x : list(jieba.cut(str(x))))
labels = labels.apply(lambda x : 1 if str(x)>="2.5" else 0)
print(time.perf_counter())

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.005 seconds.
Prefix dict has been built succesfully.


2278121.321179713
2278173.491249566


In [7]:
def time_ns(f):
    def wrapper(*args,**kwarg):
        print("%s called at %s" %(f.__name__, time.perf_counter()))
        res = f(*args, **kwarg)
        print("finished at %s" % time.perf_counter())
        return res
    return wrapper
@time_ns
def train_test_split(X, Y, train_test_ratio=0.7):
    word_idx = np.arange(0,len(X))
    np.random.shuffle(word_idx)
    bound = int(train_test_ratio*len(X))
    train_idx = word_idx[:bound]
    test_idx = word_idx[bound:]
    y = Y.values.reshape(-1,1)
    return X[train_idx],y[train_idx], X[test_idx], y[test_idx]
train_words, train_labels, test_words, test_labels \
    = train_test_split(cut_words,labels)

- [ ] why OOV? 
- [ ] how to solve it ? 
- [ ] train test 在什么时候划分? word2Vec 之前还是之后?
> 应该在word2vec之后, 词向量只能用训练集的数据进行训练.

In [9]:
vector_size = 200
word2vec = gensim.models.Word2Vec(sentences=train_words,size=vector_size)
word2vec.wv.get_vector("你")
# generate word_vector
@time_ns
def sentence2vector(sentences, word2vec, vector_size):
    sentence_vectors = []
    for s in sentences:
        word_vectors = []
        for w in s:
            try:
                vector = word2vec.wv.get_vector(w)
            except KeyError:
                vector = np.zeros([vector_size,],dtype= np.float)
            word_vectors.append(vector)
        sentence_vectors.append(np.mean(word_vectors,axis=0))
        assert len(sentence_vectors[0])==vector_size
    assert len(sentence_vectors) == len(sentences)
    return sentence_vectors
train_vectors = sentence2vector(train_words, word2vec, vector_size)
test_vectors = sentence2vector(test_words, word2vec, vector_size)

sentence2vector called at 2278225.459254632
finished at 2278240.231749165
sentence2vector called at 2278240.232069195
finished at 2278246.496163302


### fit with mean of vectors
represent sentence by mean of word vectors

In [10]:
def accuracy(y_hat, y_true, db=0.5):
    """
    :param y_hat: the soft-max value of prediction  
    :param y_true: the labels 
    :param db: decision boundary
    :return: accuracy rates
    """
    y_hat_labels = np.array(list(map(lambda x: [x[0]>=db], y_hat)))
    return np.mean(y_hat_labels==y_true)
num_epoch = 400
layer_units = [100,70,50,10,1]
#build model 
mean_graph = tf.Graph()
with mean_graph.as_default():
    global_step = tf.Variable(0)
    sent_vect = tf.placeholder(tf.float32,shape=[None, vector_size])
    sent_score = tf.placeholder(tf.float32, shape=[None,1])
    
    with tf.name_scope("layer1"):
        w_1 = tf.Variable(tf.truncated_normal([vector_size, layer_units[0]]) ,name="w")
        b_1 = tf.Variable(tf.zeros([layer_units[0]]),name="b")
        unit_1 = tf.nn.relu(tf.matmul(sent_vect,w_1) + b_1)
    with tf.name_scope("layer2"):
        w_2 = tf.Variable(tf.truncated_normal([layer_units[0], layer_units[1]]) ,name="w")
        b_2 = tf.Variable(tf.zeros([layer_units[1]]),name="b")
        unit_2 = tf.nn.relu(tf.matmul(unit_1,w_2) + b_2)
    with tf.name_scope("layer3"):
        w_3 = tf.Variable(tf.truncated_normal([layer_units[1], layer_units[2]]) ,name="w")
        b_3 = tf.Variable(tf.zeros([layer_units[2]]),name="b")
        unit_3 = tf.nn.relu(tf.matmul(unit_2,w_3) + b_3)
    with tf.name_scope("layer4"):
        w_4 = tf.Variable(tf.truncated_normal([layer_units[2], layer_units[3]]) ,name="w")
        b_4 = tf.Variable(tf.zeros([layer_units[3]]),name="b")
        unit_4 = tf.nn.relu(tf.matmul(unit_3,w_4) + b_4)
    with tf.name_scope("layer5"):
        w_5 = tf.Variable(tf.truncated_normal([layer_units[3], layer_units[4]]) ,name="w")
        b_5 = tf.Variable(tf.zeros([layer_units[4]]),name="b")
        logits = tf.matmul(unit_4,w_5) + b_5
        predictions = tf.nn.sigmoid(logits)
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,labels=sent_score))
    optimizer = tf.train.AdagradOptimizer(0.001).minimize(loss,global_step)
with tf.Session(graph=mean_graph) as sess:
    tf.initialize_all_variables().run()
    counter = 0
    for epoch in tqdm(range(num_epoch)):
        counter += 1
        feed_dict = {sent_vect:train_vectors, sent_score:train_labels}
        _, l ,train_pred = sess.run([optimizer, loss, predictions], feed_dict=feed_dict)
        if counter %10 == 0:
            train_acc = accuracy(train_pred,train_labels)
            print("train acc %s after %s" %(train_acc, epoch))
            print("loss %s after %s" %(l, epoch))
        if counter %100 == 0:
            test_pred = sess.run([predictions], feed_dict = {sent_vect:test_vectors, sent_score:test_labels})
            test_acc = accuracy(test_pred, test_labels)
            print("test acc %s after %s" %(test_acc, epoch))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use `tf.global_variables_initializer` instead.
train acc 0.6521385218004119 after 9
loss 39.675556 after 9
train acc 0.654760799139019 after 19
loss 32.97599 after 19
train acc 0.6612181570853387 after 29
loss 28.239458 after 29
train acc 0.6685386813222833 after 39
loss 24.590548 after 39
train acc 0.6754767901140144 after 49
loss 21.763584 after 49
train acc 0.6815790480040645 after 59
loss 19.537056 after 59
train acc 0.6875501920271843 after 69
loss 17.728819 after 69
train acc 0.6923467743257197 after 79
loss 16.233412 after 79
train acc 0.6948707162641289 after 89
loss 14.966144 after 89
train acc 0.6967500150234639 after 99
loss 13.890141 after 99
train acc 0.6983506968155719 after 109
loss 12.9628105 after 109
train acc 0.6989516353723361 afte

100%|██████████| 400/400 [05:42<00:00,  1.08s/it]


## Implements on Compute Graph
### concepts inventory

node: 单个节点代表某层参数/计算逻辑, 前向反馈过程中的一个计算结果或参数.

attri: data, forward_nodes, backward_nodes, value, gradients

从forward 角度, 多对一, 而没有一对多( lstm除外 )

method: forward: compute value , call forward method of nodes in the forward list 
        backward: compute gradient, call backward method of nodes in the backward list
       

In [13]:
from abc import abstractmethod 
class Nodes:
    def __init__(self, inputs, data):
        self.inputs = inputs
        self.data = data
        self.outputs = []
        self.gradients = {}
        if inputs!= None:
            for i in inputs:
                i.outputs.append(self)
    # forward 和 backward 时的顺序应该先进行拓扑排序.
    @abstractmethod
    def forward(self):
        pass
    
    @abstractmethod
    def backward(self):
        pass

class Paras(Nodes):        
    def forward(self):
        self.value = self.data
    
    def backward(self):
        for o in self.outputs:
            if self.gradients.get(self):
                self.gradients[self] += o.gradient[self]
            else:
                self.gradients[self] = o.gradient[self]
        self.gradients[self] /= len(self.outputs)
class Inputs(Nodes):
    def forward(self):
        self.value = self.data
        
    
    def backward(self):
        pass

class Linear(Nodes):
    def __init__(self,x_node,w_node, b_node):
        self.x_node = x_node 
        self.w_node = w_node
        self.b_node = b_node 
    
    def forward(self):
        self.value = np.dot(self.x_node, self.w_node) + self.b_node
    
    def backward(self):
        for o in self.outputs:
            grad = o.gradients[self]
            self.gradients[self.w_node] = np.dot(self.x_node.value.T, grad)
            self.gradients[self.b_node] = np.mean(grad,axis=0, keepdims= True)
            
    class Sigmoid(Nodes):
        def __init__(self, node):
            Nodes.__init__(self, [node])
            self.x_node = node
        
        def _sigmoid(self, x):
            return 1. / (1 + np.exp(-1 * x))
        
        def forward(self):
            self.value = self._sigmoid(self.x_node.value)
        
        def backward(self):
            y = self.value
            
            self.partial = y * (1 - y)
            
            for n in self.outputs:
                grad_cost = n.gradients[self]
                
                self.gradients[self.x_node] = grad_cost * self.partial
                
    class MSE(Nodes):
        def __init__(self, y_true, y_hat):
            self.y_true_node = y_true
            self.y_hat_node = y_hat
            Nodes.__init__(self, inputs=[y_true, y_hat])
        
        def forward(self):
            y_true_flatten = self.y_true_node.value.reshape(-1, 1)
            y_hat_flatten = self.y_hat_node.value.reshape(-1, 1)
            
            self.diff = y_true_flatten - y_hat_flatten
            
            self.value = np.mean(self.diff**2)
            
        def backward(self):
            n = self.y_hat_node.value.shape[0]
            
            self.gradients[self.y_true_node] = (2 / n) * self.diff
            self.gradients[self.y_hat_node] =  (-2 / n) * self.diff